In [1]:
import os
import pandas as pd
import numpy as np
import warnings
from tqdm import tqdm
import re
import statsmodels as  sm

# 实验一：

## 1. 整理公司DSE資料

### 1.1 读取高管数据
#### 1.1.1 处理格式问题
- 由于极少数据格式对应错误，无法读取，因此略过
  - ![](2023-10-24-10-17-04.png)

In [2]:
gaoguan = pd.read_csv('./高管个人资料文件/CG_Director.csv',
                      encoding='utf-8',
                      dtype={'Stkcd':str, 'Symbol':str}, 
                      sep=',', 
                      skiprows=[40047,40063,73958,144620])
# dtype = {'Stkcd':str, 'Symbol':str} 用于将Stkcd和Symbol转换为str类型，否则会出现科学计数法; sep = ',' 用于指定分隔符为逗号
# 忽略不合适的行
warnings.filterwarnings('ignore')
gaoguan.columns

Index(['Stkcd', 'Reptdt', 'D0101b', 'PersonID', 'D0201a', 'D0201b', 'D0301b',
       'D0401b', 'D0501b', 'D0601b', 'D0701b', 'D0702b', 'D0801c', 'D0901b',
       'D1001b', 'D1002b', 'D1101b', 'D1201b', 'YearBeginningHoldShares',
       'ChangeReason', 'ShareHoldingStyle', 'StockOptionsNumber',
       'ConditionalSharesNumber', 'ExercisedNumber', 'AwardPrice',
       'UnlockNumber', 'EndPrice', 'ExercisedPrice', 'EndReferMarketValue',
       'EndSalaryAMarketValue', 'D1301b', 'PositionID'],
      dtype='object')

####  1.1.1 选取部分列并缺失值处理

In [3]:

#* 重命名列名
dict_col={
'Stkcd':'Stkcd',
'Reptdt':'date',
'D0101b':'姓名',
'PersonID':'PersonID',
'D0201a':'职务类别',
'D0301b':'性别',
'D0401b':'年龄',
'D0501b':'教育背景',
'D0701b':'任职开始日期',
'D0702b':'任职结束日期',
'D1001b':'报告期报酬总额',
'StockOptionsNumber':'股票期权数量',
'ExercisedNumber':'股票期权已行权数量',
'ExercisedPrice':'股票期权行权价',
'ConditionalSharesNumber':'被授予的限制性股票数量',
'AwardPrice':'限制性股票授予价格',
'EndPrice':'期末股价',
'D1101b':'年末持股数',
}

In [4]:
gaoguan.rename(columns=dict_col, inplace=True)

In [5]:
gaoguan = gaoguan.loc[:,dict_col.values()] # type: ignore 
gaoguan.head()

,Stkcd,date,姓名,PersonID,职务类别,性别,年龄,教育背景,任职开始日期,任职结束日期,报告期报酬总额,股票期权数量,股票期权已行权数量,股票期权行权价,被授予的限制性股票数量,限制性股票授予价格,期末股价,年末持股数
0,000001,2021-12-31,谢永林,30182250,1100000000,男,53.0,5.0,2016-12-10,NaN,NaN,0.0,NaN,NaN,0.0,NaN,16.48,26700.0
1,000001,2021-12-31,胡跃飞,30149118,10300000N0,男,59.0,4.0,"2007-12-19,2016-12-10",NaN,3879500.0,0.0,NaN,NaN,0.0,NaN,16.48,104104.0
2,000001,2021-12-31,陈心颖,30285637,1000000000,女,44.0,4.0,2014-01-21,NaN,NaN,0.0,NaN,NaN,0.0,NaN,16.48,0.0
3,000001,2021-12-31,姚波,3052624,1000000000,男,50.0,4.0,2010-06-17,NaN,NaN,0.0,NaN,NaN,0.0,NaN,16.48,0.0
4,000001,2021-12-31,蔡方方,30285638,1000000000,女,47.0,4.0,2014-01-21,NaN,NaN,0.0,NaN,NaN,0.0,NaN,16.48,0.0


In [6]:

#* 清除空缺值
gaoguan['教育背景'].fillna(6, inplace=True) # 6代表其他

for col in gaoguan.columns:
    gaoguan[col].fillna(0, inplace=True) # 0代表缺失值

#gaoguan.to_csv(r'./数据处理中间结果/高管个人资料文件.csv', index=False)

### 1.2 读取、整理其余数据

In [7]:

#* 兼任信息文件
jianren=pd.read_csv(r'./兼任信息文件/CG_Rzgddw.csv')
dict_col={
    'Stkcd':'Stkcd',
    'Reptdt':'date',
    'D0101b':'姓名',
    'PersonID':'PersonID',
    'ConcurrentInstitutionID':'兼任机构ID',
    'D0105b':'职务名称', # 是否包含"人大"、'政协'
    }
jianren.rename(columns=dict_col, inplace=True)
jianren=jianren.loc[:, dict_col.values()] # type: ignore

In [8]:
# jianren.to_csv(r'./数据处理中间结果/兼任信息文件.csv', index=False)
jianren.head()

,Stkcd,date,姓名,PersonID,兼任机构ID,职务名称
0,1,2021-12-31,谢永林,30182250,103379.0,"执行董事,总经理,联席首席执行官,党委副书记"
1,1,2021-12-31,陈心颖,30285637,103379.0,"执行董事,联席首席执行官,常务副总经理"
2,1,2021-12-31,陈心颖,30285637,108216.0,非执行董事
3,1,2021-12-31,姚波,3052624,103379.0,"联席首席执行官,常务副总经理,首席财务官,执行董事"
4,1,2021-12-31,姚波,3052624,108216.0,非执行董事


In [9]:

#* 董监高特征
#! 跳过错误的行
dse=pd.read_csv(r'./董监高个人特征文件/TMT_FIGUREINFO.csv',
                dtype={'Resume':str,'OtherCo':str},
                skiprows=[40693,40699,74614,148282])
dict_col={
    'Stkcd':'Stkcd',
    'Reptdt':'date',
    'Name':'姓名',
    'PersonID':'PersonID',
    'Gender':'性别',
    'Age':'年龄',
    'TMTP':'高管职务类别',
    'CTB':'董事会职务类别',
    'IsIdirecotr':'是否独立董事',
    'ServicePosition':'在职职务',
    'Funback':'职业背景',
    'University':'毕业院校',
    'Degree':'学历',
    'Major':'专业',
    'FinBack':'金融背景', # 1=监管部门
    'IsDuality':'两职兼任',
    }
dse.rename(columns=dict_col, inplace=True)
dse=dse.loc[:, dict_col.values()] # type: ignore

dse.to_csv(r'./数据处理中间结果/董监高个人特征.csv', index=False)

In [10]:

#* 董事长、CEO变更
ceo=pd.read_csv(r'./董事长与总经理变更文件/CG_Ceo.csv')
dict_col={
    'Stkcd':'Stkcd',
    'Annodt':'date',
    'Name':'姓名',
    'PersonID':'PersonID',
    'Changtyp':'离任继任',
    'Position':'变更职位',
    'Chgdt':'离职继任日期',
    'Edca':'教育背景',
    'Cirplurm':'兼任情况', # 离任时，此字段无意义
    }
ceo.rename(columns=dict_col, inplace=True)
ceo=ceo.loc[:, dict_col.values()] # type: ignore
ceo['year']=ceo['date'].apply(lambda x: int(x[:4]))


In [11]:
ceo.head()

,Stkcd,date,姓名,PersonID,离任继任,变更职位,离职继任日期,教育背景,兼任情况,year
0,6,2022-11-16,赵宏伟,30366943,1,1,2022-11-16,3.0,NaN,2022
1,12,2022-08-16,何进,30395774,2,2,2022-08-15,4.0,1.0,2022
2,12,2022-08-16,王健,30330803,1,2,2022-08-15,NaN,NaN,2022
3,17,2022-11-29,李海,3044698,1,1,2022-11-27,3.0,NaN,2022
4,17,2022-11-29,王胜洪,30666305,2,1,2022-11-28,NaN,1.0,2022


In [13]:

# 若职位为董事长，或者CEO兼任董事长，则认为是董事长
def find_ceo(row):
    flag=0
    if row['变更职位']==1:
        flag=1
    elif (row['变更职位']==2) and (row['兼任情况']==2):
        flag=1
    return flag
ceo['ceo']=ceo.apply(lambda x: find_ceo(x), axis=1)
# 只保留董事长
ceo=ceo.loc[ceo['ceo']==1, :].reset_index(drop=True)

# 规范personID
ceo['PersonID'].fillna('', inplace=True)
ceo['PersonID']=ceo['PersonID'].astype(str)
ceo['董事长id']=ceo.apply(lambda x: str(x['Stkcd'])+'-'+x['姓名']+'-'+x['PersonID'], axis=1)

# 区分 任职日期、离任日期
ceo['任职日期']=''
ceo['离任日期']=''
ceo.loc[ceo['离任继任']==1, '离任日期']=ceo['离职继任日期']
ceo.loc[ceo['离任继任']==2, '任职日期']=ceo['离职继任日期']


# 按姓名分组，并按年份升序排序
ceo = ceo.sort_values(by=['董事长id', 'date'])


for index, row in ceo.iterrows():
    name = row['董事长id'] 
    # 如果是离任状态
    if row['离任继任'] == 1:
        # 查找之前的上任记录以更新上任日期
        previous = ceo[(ceo['董事长id'] == name) & (ceo['离任继任'] == 2) & (ceo.index < index)]
        if previous.shape[0] > 0:
            ceo.loc[ceo.index==index, '任职日期'] = list(previous['任职日期'])[-1]
    # 如果是继任状态
    elif row['离任继任'] == 2:
        # 查找后面的离任记录以更新离任日期
        follow = ceo[(ceo['董事长id'] == name) & (ceo['离任继任'] == 1) & (ceo.index > index)]
        if follow.shape[0] > 0:
            ceo.loc[ceo.index==index, '离任日期'] = list(follow['离任日期'])[0]

# 删除重复记录
ceo = ceo.drop_duplicates(['董事长id','任职日期','离任日期'])
# 补充缺失值
ceo['教育背景'].fillna(6, inplace=True)
ceo['兼任情况'].fillna(1, inplace=True)

# 将日期列转换为datetime类型
ceo['任职日期'] = pd.to_datetime(ceo['任职日期'])
ceo['离任日期'] = pd.to_datetime(ceo['离任日期'])

ceo=ceo.loc[:, ['Stkcd', 'date', '董事长id', 'PersonID','任职日期', '离任日期','教育背景','兼任情况']]

#ceo.to_csv(r'./数据处理中间结果/董事长任期数据.csv', index=False)

In [14]:

#* 违规数据
violate=pd.read_csv(r'./违规信息总表/STK_Violation_Main.csv',skiprows=[1247,1363,1550,1551,2039,2248,2258,2283])
dict_col={
    'Symbol':'Stkcd',
    'DeclareDate':'date',
    'CoFullName':'违规公司名称',
    'ViolationType':'违规类型',
    'ViolationYear':'实际违规的年份',
    'Activity':'处罚公告内容',
    'IsViolated':'上市公司是否违规',
    }

violate.rename(columns=dict_col, inplace=True)
violate=violate.loc[:, dict_col.values()] # type: ignore

violate[violate['实际违规的年份'].isnull()].shape


(123, 7)

In [15]:
# 只保留违规的记录
violate=violate.loc[violate['上市公司是否违规']=='Y', :]
violate['year']=violate['date'].apply(lambda x: x[:4])
violate.loc[violate['实际违规的年份'].isnull(), '实际违规的年份']=violate['date']

In [16]:

df = []
for ix,row in violate.iterrows():
    company = row['Stkcd']
    name = row['违规公司名称']
    years = re.split(r'[,;]', row['实际违规的年份']) # 以逗号或分号分割 
    vio_type = re.split(r'[,;]', row['违规类型'])
    events = row['上市公司是否违规']
    content = row['处罚公告内容']
    
    # 对于每个年份，创建一个新的数据行并添加到DataFrame中
    for j in range(len(years)):
        new_row = {'Stkcd': company,'违规公司名称': name,
                   '实际违规的年份': years[j], '上市公司是否违规': events,
                   '违规类型':vio_type[j], '处罚公告内容':content}
        df.append(new_row)

df = pd.DataFrame(df)

# 整理格式
def norm_year(string):
    if string=='N/A':
        return ''
    elif len(string)>4:
        return string[:4]
    else:
        return string
df['实际违规的年份']=df['实际违规的年份'].apply(lambda x: norm_year(x))
df=df.loc[df['实际违规的年份']!='', :].drop_duplicates()
df.rename(columns={'实际违规的年份':'year', '上市公司是否违规':'公司违规'}, inplace=True)
df['year']=df['year'].astype(int)
df.sort_values(['Stkcd', 'year'], inplace=True)


# 同一年、同一个公司可能有多个名称，只保留最后一个名称
name = df.loc[:, ['Stkcd', '违规公司名称', 'year']].drop_duplicates(['Stkcd', 'year'])

def sel_vio(string):
    tmp=['虚构利润',' 虚列资产','虚假记载(误导性陈述)','重大遗漏','披露不实(其它)','欺诈上市',
         '出资违规','擅自改变资金用途','占用公司资产','一般会计处理不当','偷税','虚开普通发票',
         '虚开增值税专用发票或者虚开用于骗取出口退税、抵扣税款的其他发票',
         '私自印制、伪造、变造发票，非法制造发票防伪专用品，伪造发票监制章']
    flag = 0
    for i in tmp:
        if i in string:
            flag += 1
    return flag
df['违规次数']=df['违规类型'].apply(lambda x: sel_vio(x))

# 计算违规次数
df2=df.groupby(['Stkcd', 'year'])['违规次数'].aggregate(sum)
df2=pd.DataFrame(df2)
df2=df2.merge(name, on=['Stkcd', 'year'], how='left')

df2.to_csv(r'./数据处理中间结果/违规数据.csv', index=False)

In [17]:

#* 上市公司
firm = pd.read_excel(r'./上市公司基本信息年度表/上市公司基本信息年度表(联表查询).xlsx')
dict_col={
    'Symbol':'Stkcd',
    'EndDate':'date',
    'ListedCoID':'机构ID',
    'IndustryName': '行业名称',
    'IndustryCode': '行业代码',
    'FullName': '中文全称',
    'LegalRepresentative': '法人代表',
    'EstablishDate': '公司成立日期',
    'RegisterCapital': '注册资本',
    'LISTINGDATE': '首次上市日期',
    'PROVINCE': '所属省份',
    'CITY': '所属城市',
    'Nshrttl': '总股数',
    'EquityNature': '股权性质',
    'LISTINGSTATE': '上市状态',
    }
firm.columns = [i.split('.')[1] for i in firm.columns]
firm.rename(columns=dict_col, inplace=True)
firm['Stkcd']=firm['Stkcd'].apply(lambda x: str(x).rjust(6,'0'))
firm = firm.loc[:, dict_col.values()] # type: ignore
firm.to_csv(r'./数据处理中间结果/公司基本信息.csv', index=False)

In [18]:

#* 关联交易
rpt_trans=pd.read_csv(r'./关联交易情况文件/RPT_Operation.csv',
                      dtype={'Notes':str,'Trddt':str},
                      on_bad_lines='skip'
)
dict_col={
    'Stkcd':'Stkcd',
    'Repart':'关联方',
    'Relation':'关联关系',
    'Annodt':'公告日期',
    'Trasub':'交易性质',
    'Repat':'关联交易事项分类',
    'Trddt':'交易日期',
    }

rpt_trans.rename(columns=dict_col, inplace=True)
rpt_trans=rpt_trans.loc[:, dict_col.values()] # type: ignore

rpt_trans['交易日期'].dropna(inplace=True)

# 规范关联方名称
rpt_trans['关联方']=rpt_trans['关联方'].apply(lambda x: x.replace('及其子公司',''))
rpt_trans['关联方']=rpt_trans['关联方'].apply(lambda x: x.replace('及其关联方',''))
rpt_trans['关联方']=rpt_trans['关联方'].apply(lambda x: x.replace('及近亲属',''))

# 将多个关联方交易拆分为多行，每一行一个关联方
# 创建一个空的DataFrame来存储拆分后的数据
df = []
# 遍历原始数据列表
for ix,row in rpt_trans.iterrows():
    value1 = row['Stkcd']
    value2 = row['关联关系']
    value3 = row['公告日期']
    value4 = row['交易日期']
    value6 = row['交易性质']
    value7 = row['关联交易事项分类']
    rpt_firms = re.split(r'[,]', row['关联方'])
    
    # 对于每个年份，创建一个新的数据行并添加到DataFrame中
    for j in rpt_firms:
        new_row = {'Stkcd': value1,
                   '关联方':j,
                   '关联关系': value2,
                   '公告日期': value3,
                   '交易日期': value4,
                   '交易性质':value6,
                   '关联交易事项分类':value7,
                   }
        df.append(new_row)

df = pd.DataFrame(df)

df.to_csv(r'./数据处理中间结果/关联交易情况文件.csv', index=False)

100%|██████████| 377388/377388 [00:21<00:00, 17386.02it/s]


## 2. 整理公司财务、股价数据

In [19]:

#* 1. 比率结构
ratio = pd.read_csv(r'./比率结构/FI_T3.csv', dtype={'Stkcd':str})
dict_col={
'Stkcd':'Stkcd',
'Accper':'统计截止日期',
'F030101A':'流动资产比率',
'F030201A':'现金资产比率',
'F032001A':'少数股东权益占比',
'F031101A':'所有者权益比率',
'F031501A':'流动负债比率',
'F033101B':'净利润综合收益占比',
}
ratio.rename(columns=dict_col, inplace=True)
ratio = ratio.loc[:, dict_col.values()] # type: ignore
ratio['year'] = ratio['统计截止日期'].apply(lambda x: int(x[:4]))
ratio.drop('统计截止日期', axis=1, inplace=True)
ratio.drop_duplicates(['Stkcd','year'], inplace=True)
ratio.to_csv(r'./数据处理中间结果/比率结构.csv', index=False)

In [20]:

#* 2. 相对价值指标
value = pd.read_csv(r'./相对价值指标/FI_T10.csv', dtype={'Stkcd':str})
dict_col={
    'Stkcd':'Stkcd',
    'Accper':'统计截止日期',
    'F100102B':'PE', 
    'F100401A':'PB',
    'F100801A':'市值',
    'F100901A':'托宾Q值1',
    'F100902A':'托宾Q值2',
    'F101001A':'账面市值比1',
    }
'''
F100102B [市盈率（PE）2] - 计算公式为：今收盘价当期值/（调整因子*净利润当期值/实收资本本期期末值）；
当分母未公布或为零或小于零时，以NULL表示；
调整因子：调整因子和（利润表科目）会计期间相关（即是：开始日期、截止日期的月份差额）；即是：12/(间隔月份)。比如，一季度为：12/3；前6个月：12/6,；1到9月份：12/9，全年：1；下同
F100901A [托宾Q值A] - 计算公式为：市值A/资产总计；当分母未公布或为零或小于零时，以NULL表示。
F100902A [托宾Q值B] - 计算公式为：市值A/（资产总计—无形资产净额—商誉净额）；当分母未公布或为零或小于零时，以NULL表示。
'''

value.rename(columns=dict_col, inplace=True)
value = value.loc[:, dict_col.values()] # type: ignore
value['year'] = value['统计截止日期'].apply(lambda x: int(x[:4]))
value.drop('统计截止日期', axis=1, inplace=True)
value.drop_duplicates(['Stkcd','year'], inplace=True)
value.to_csv(r'./数据处理中间结果/相对价值指标.csv', index=False)

In [24]:

#* 3.盈利能力
# 盈利能力
profit = pd.read_csv(r'./盈利能力/FI_T5.csv', dtype={'Stkcd':str})
dict_col={
    'Stkcd':'Stkcd',
    'Accper':'统计截止日期',
    #'F050201B':'ROA1',
    'F050202B':'ROA2',
    #'F050203B':'ROA3',
    'F050502B':'ROE2',
    #'F050503B':'ROE3',
    'F051801B':'管理费用率',
    'F051701B':'销售费用率',
    'F051201B':'ROIC',
    'F053301B':'营业毛利率'
    }
'''
F050202B [总资产净利润率(ROA)B] -
计算公式为：净利润／总资产平均余额；当分母未公布或为零时，以NULL表示：总资产平均余额=资产合计期末余额+资产合计期初余额）/2
F050203B [总资产净利润率(ROA)C] - 
计算公式为：净利润／总资产平均余额；当分母未公布或为零时，以NULL表示；总资产平均余额=资产合计期末余额+资产合计上年期末余额）/2
F051201B [投入资本回报率（ROIC）] - 
计算公式为：（净利润+财务费用） ／（资产总计-流动负债+应付票据+短期借款+一年内到期的非流动负债）。当分母未公布或为零或小于零时，以NULL表示。
F053301B [营业毛利率] - 计算公式为：07年后为：（营业收入—营业成本）／营业收入；当分母未公布或为零时，以NULL表示
'''
profit.rename(columns=dict_col, inplace=True)
profit = profit.loc[:, dict_col.values()] # type: ignore
profit['year'] = profit['统计截止日期'].apply(lambda x: int(x[:4]))
profit.drop('统计截止日期', axis=1, inplace=True)
profit.drop_duplicates(['Stkcd','year'], inplace=True)
value.to_csv(r'./数据处理中间结果/盈利能力.csv', index=False)

In [25]:

#* 4. 股票数据
stock = pd.read_excel(r'./日个股回报率文件/TRD_Dalyr.xlsx',header = 1, skiprows=[0,2])
stock.to_csv(r'./数据处理中间结果/日个股回报率.csv', index=False)

In [ ]:
## 3. 用中间结果构建数据集